In [123]:
import pprint

import language_processing as lang_process
import parse_institutional_data as parse_data
import entity_processing as entity_proc
import process_search_terms as process
import json
import os

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vonbecker/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/vonbecker/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/vonbecker/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vonbecker/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/vonbecker/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [124]:
from importlib import reload
reload(lang_process);
reload(parse_data);
reload(process);
reload(entity_proc)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vonbecker/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/vonbecker/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/vonbecker/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vonbecker/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/vonbecker/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


<module 'entity_processing' from '/Users/vonbecker/ML_Bootcamp_code/Docent_AI/data/training_data/analysis/entity_processing.py'>

In [125]:
# read in and pre-process the exhibit keywords and phenomena

path = '../get_institutional_data/data/raw/exhibits.json'
phenomena = parse_data.get_keywords(path, 'phenomena')
proc_phenomena = [process.preprocess(phenomenon, 'phenomenon') for phenomenon in phenomena]
keywords = parse_data.get_keywords(path, 'keywords')
proc_keywords = [process.preprocess(keyword, 'keyword') for keyword in keywords]

In [126]:
# join preprocessed keywords and phenomena, keep unique elements

all_search_terms = proc_phenomena + proc_keywords
unique_search_terms = []
[unique_search_terms.append(x) for x in all_search_terms if x not in unique_search_terms];

In [136]:
# create search_terms.json, containing primary and related search terms
# create primary_with_syns.json, containing primary search terms and corresponding word senses
# NEED TO MATCH ON WORD?

search_terms = [process.get_search_terms(term) for term in unique_search_terms]
primary_with_syns = [process.get_all_data(term) for term in unique_search_terms]

path = '../get_institutional_data/data/'
if not os.path.exists(path):
    os.makedirs(path)

primary_with_syns_filepath = os.path.join(path, 'primary_with_syns.json')
if os.path.exists(primary_with_syns_filepath):
    os.remove(primary_with_syns_filepath)

with open(primary_with_syns_filepath, 'w') as outfile:
    outfile.write(json.dumps(primary_with_syns, indent=2))

search_terms_filepath = os.path.join(path, 'search_terms.json')
if os.path.exists(search_terms_filepath):
    os.remove(search_terms_filepath)

with open(search_terms_filepath, 'w') as outfile:
    outfile.write(json.dumps(search_terms, indent=2))

In [137]:
# create search_term_word_sense.json, containing

import save_word_sense as save
reload(save)

path = '../get_institutional_data/data/'

word_sense = save.save_word_senses(path, 252)

In [138]:
pp.pprint(word_sense)

[{'data': [{'sense': 'water.n.01', 'word': 'water'}],
  'primary': 'water',
  'timestamp': '11-01-2022_052844'},
 {'data': [{'sense': 'water.n.01', 'word': 'water'}],
  'primary': 'water',
  'timestamp': '11-01-2022_052844'},
 {'data': [{'sense': 'water.n.01', 'word': 'water'}],
  'primary': 'water',
  'timestamp': '11-01-2022_052844'},
 {'data': [{'sense': 'water.n.01', 'word': 'water'}],
  'primary': 'water',
  'timestamp': '11-01-2022_052844'},
 {'data': [{'sense': 'water.n.01', 'word': 'water'}],
  'primary': 'water',
  'timestamp': '11-01-2022_052844'},
 {'data': [{'sense': 'water.n.01', 'word': 'water'}],
  'primary': 'water',
  'timestamp': '11-01-2022_052844'},
 {'data': [{'sense': 'water.n.01', 'word': 'water'}],
  'primary': 'water',
  'timestamp': '11-01-2022_052844'},
 {'data': [{'sense': 'water.n.01', 'word': 'water'}],
  'primary': 'water',
  'timestamp': '11-01-2022_052844'},
 {'data': [{'sense': 'water.n.01', 'word': 'water'}],
  'primary': 'water',
  'timestamp': '11-0

In [121]:
# clean up existing output files
#

from difflib import SequenceMatcher

# simarilty score for strings a and b
# score in [0,1]
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


# initialize paths for input and output data sources
path = '../get_institutional_data/data/'
path_to_labels = os.path.join(path, 'labeled_search_phrases/')

# load all search terms
search_terms_filepath = os.path.join(path, 'primary_with_syns.json')
with open(search_terms_filepath, 'r') as infile:
    search_terms = json.load(infile)

# initialize output data
all_data = []

# iterate through json files in 'labeled_search_phrases/'
for filename in os.listdir(path_to_labels):
    # ignore hidden files
    if filename.startswith('.'):
        continue

    filepath = os.path.join(path_to_labels, filename)

    # extract starting index of data from filename
    start_ind = int(filename.split('_')[0])

    # load file contents into the data object
    with open(filepath, 'r') as infile:
        data = json.load(infile)

    # initialize output for this file
    this_file = []
    counter = 0

    # iterate through items (phrases) in data
    for phrase in data:
        ind = start_ind + counter
        num_words = len(search_terms[ind]['primary'])
        orig_words = [search_terms[ind]['primary'][n][0] for n in range(num_words)]
        this_phrase = []
        for word in phrase:
            word_text = word['word']
            new_word = {}
            match_ind = None
            for orig_word in orig_words:
                if compare(word_text, orig_word):
                    match_ind = orig_words.index(orig_word)
                try:
                    new_word['orig_word'] = orig_words[match_ind]
                except IndexError or TypeError:
                    pass
            new_word['sense'] = word['sense']
            this_phrase.append(new_word)
        counter += 1
        this_file.append(this_phrase)
    all_data.append(this_file)

TypeError: list indices must be integers or slices, not NoneType

In [116]:
pp.pprint(all_data)

[]
